In [ ]:
!pip install langchain boto3
!pip install transformers
!pip install sentence_transformers

In [ ]:
!pip install langchain==0.0.224

In [ ]:
!pip install boto3 --upgrade

In [ ]:
!pip install ~/SageMaker/botocore-1.29.162-py3-none-any.whl
!pip install ~/SageMaker/boto3-1.26.162-py3-none-any.whl

In [ ]:
!pip install wikipedia

### Pre-requisites

Deploy Falcon/Llama model as a Sagemaker Endpoint prior to running this example 

In [ ]:
import boto3
import langchain
from langchain.retrievers import AmazonKendraRetriever
from typing import List
from typing import Dict
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
#from langchain import SagemakerEndpoint, LLMChain
from langchain import LLMChain
from langchain.llms.bedrock import Bedrock
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json

from langchain.docstore.document import Document
from langchain.memory import ConversationBufferWindowMemory
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

from langchain.tools import BaseTool, StructuredTool, Tool, tool
from langchain.prompts import StringPromptTemplate
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, OutputParserException

import re

In [ ]:
class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
#        return response_json["vectors"]
        return response_json[0]["generated_text"]



content_handler = ContentHandler()


### Create the tools

We create the "tools" which will be executed by agent. In this example we create 3 tools, one for addition, another for subtraction and one for generating a random number. The input into this tool is a string in the format "x,y" for the addition/subtraction functions and a single number for the random number generator. Note that the inputs are strings and hence we can use a split function. This works for a simplistic example, but ideally the LLM should respond with a JSON object which can be parsed by this tool. 

In [ ]:
import requests
from langchain.tools import BaseTool
from bs4 import BeautifulSoup
import random

def addTwoNumbers(numberList: str) -> str:
    print("\nthe add numbers tools executed\n")
    x = numberList.split(",")
    answer = int(x[0]) + int(x[1])
    return "The sum is " + str(answer)

def subtractTwoNumbers(numberList):
    print("\nthe subtract numbers tools executed\n")
    x = numberList.split(",")
    answer = int(x[0]) - int(x[1])
    return "The difference is " + str(answer)


def generateRandomNumber(aNumber):
    #print(aNumber)
    print("\nthe random number tool executed\n")
    return "A random number is " + str(random.randint(0,int(aNumber)))

random_number_tool = Tool(
    name="RandomNumberTool",
    func = generateRandomNumber,
    description="a tool to generate random numbers"
)

addition_tool = Tool(
    name="AdditionTool",
    func = addTwoNumbers,
    description="a tool to add two numbers"
)

subtraction_tool = Tool(
    name="SubtractionTool",
    func = subtractTwoNumbers,
    description="a tool to get the difference between two numbers"
)

tools = [random_number_tool, addition_tool, subtraction_tool]

### The ReAct prompt

A prompt template is created for the few-shot-learning example. The prompts are in the format Human/Thought/Action/Action Input/Observation/Action. 

The Action Input: in the prompt is used to pass parameters to the tools and is optional if the tool doesn't require any input parameters. 


In [ ]:
falcon_template = """Human: Answer the following questions as best you can. Do not use your random number generator, always use the tools. If the human asks a question that can't be solved by a tool, say "no tool available". You have access to the following tools:

{tools}

Human: Generate a number between 0 and 100 \n Thought: I need to use a tool to generate a random number with a maximum of 100 \n Action: RandomNumberTool() \n Action Input: 100 \n Observation: A random number is 6 \n Thought: I have a random number \n Action: Finish[6] 
Human: Generate a number between 0 and 100000 \n Thought: I need to use a tool to generate a random number with a maximum of 100000 \n Action: RandomNumberTool() \n Action Input: 100000 \n Observation: A random number is 6 \n Thought: I have a random number \n Action: Finish[6] 
Human: Give me a random number with a maximum of 45 \n Thought: I need to use a tool to generate a random number with a maximum of 45 \n Action: RandomNumberTool() \n Action Input: 45 \n Observation: A random number is 9 \n Thought: I have a random number \n Action: Finish[9] 
Human: Add the numbers 10 and 5 \n Thought: I need to use a tool to add two numbers 10 and 5 \n Action: AdditionTool() \n Action Input: 10,5 \n Observation: the sum is 15 \n Thought: I have the sum \n Action: Finish[15] 
Human: Combine the numbers 10 and 5 \n Thought: I need to use a tool to add two numbers 10 and 5 \n Action: AdditionTool() \n Action Input: 10,5 \n Observation: the sum is 55 \n Thought: I have the sum \n Action: Finish[15] 
Human: Subtract the numbers 10 and 5 \n Thought: I need to use a tool to subtract two numbers 10 and 5 \n Action: SubtractionTool() \n Action Input: 10,5 \n Observation: the difference is 5 \n Thought: I have the difference \n Action: Finish[5] 
Human: get the difference between 10 and 5 \n Thought: I need to use a tool to subtract two numbers 10 and 5 \n Action: SubtractionTool() \n Action Input: 10,5 \n Observation: the difference is 5 \n Thought: I have the difference \n Action: Finish[5] 
Human: get the difference between 10 and 15 \n Thought: I need to use a tool to subtract two numbers 10 and 15 \n Action: SubtractionTool() \n Action Input: 10,15 \n Observation: the difference is -5 \n Thought: I have the difference \n Action: Finish[-5] 
Human: get the difference \n \n Thought: I need to use a tool to subtract two numbers \n Action: SubtractionTool() \n Action Input: none \n Observation: the difference is 0 \n Thought: I have the difference \n Action: Finish[0] 
Human: get the difference \n \n Thought: I need to use a tool to add two numbers \n Action: AdditionTool() \n Action Input: none \n Observation: the sum is 0 \n Thought: I have the sum \n Action: Finish[0] 
Human: get the difference of 9 \n \n Thought: I need to use a tool to subtract two numbers \n Action: SubtractionTool() \n Action Input: none \n Observation: the difference is 0 \n Thought: I have the difference \n Action: Finish[0] 
Human: get the sum of 10 \n \n Thought: I need to use a tool to add two numbers \n Action: AdditionTool() \n Action Input: none \n Observation: the sum is 0 \n Thought: I have the sum \n Action: Finish[0] 

Human: {input}
{agent_scratchpad}"""

### The Prompt Template

Define a prompt template that wraps the prompt and injects the input and tools into the prompt that is fed to the LLM

In [ ]:
from typing import Callable


# Set up a prompt template
class CustomPromptTemplate(StringPromptTemplate):
    # The template to use
    template: str
    ############## NEW ######################
    # The list of tools available
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        ############## NEW ######################
        #tools = self.tools_getter(kwargs["input"])
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a tools variable from the list of tools provided
        #kwargs["tools"] = "\n".join(
        #    [f"{tool.name}: {tool.description}" for tool in tools]
        #)
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in tools])
        return self.template.format(**kwargs)

falcon_prompt = CustomPromptTemplate(
    template=falcon_template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"],
)

### Output Parser

The agent invokes the Output parser once it receives a response from the LLM. The Output parser is responsible for examining the output and determining the tool to use and the parameters to send to the tool. In the implementation below, we use regex to extract the Action and Action Inputs. An instance of AgentAction is returned that contains the tool to be executed and the parameters for the tool. In the case where the agent has completed execution, an AgentFinish object is returned to the Agent. 

Note that since regex is being used, there is tight coupling between the prompt template and agent action/action input and any change in the prompts for the Action and Action Input should be reflected in this class. Its important to set the temperature=0 to ensure that the model only returns the outputs that are expected in the Output parser

In [ ]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Finish" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Finish")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        #regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        regex=r"Action:\s([a-zA-Z]*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        #print("the action is")
        #print(action)
        
        tool_input= ""
        
        match action:
            case "RandomNumberTool" :
                #Input:\s([0-9]*)
                funcParameterRegex = r"Input:\s([0-9]*)"
                funcMatch = re.search(funcParameterRegex, llm_output, re.DOTALL)
                if not funcMatch:
                    parameter = 100
                else:
                    parameter = funcMatch.group(1).strip()
                #print("func parameter is")
                #print(parameter)
                tool_input = parameter

            case "AdditionTool":
                #Input:\s([0-9]*)\,([0-9]*)
                addParamsRegex = r"Input:\s([0-9]*)\,([0-9]*)"
                addMatch = re.search(addParamsRegex, llm_output, re.DOTALL)
                parameter = []
                if not addMatch:
                    tool_input = "0,0"
                else:
                    tool_input = addMatch.group(1).strip() + "," + addMatch.group(2).strip()
                
            
            case "SubtractionTool":
                #Input:\s([0-9]*)\,([0-9]*)
                addParamsRegex = r"Input:\s([0-9]*)\,([0-9]*)"
                addMatch = re.search(addParamsRegex, llm_output, re.DOTALL)
                if not addMatch:
                    tool_input = "0,0"
                else:
                    tool_input = addMatch.group(1).strip() + "," + addMatch.group(2).strip()

        #action_input = match.group(2)
        # Return the action and action input
        #action = action + "Tool"
        #print (action)
        return AgentAction(
            #tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output
            tool=action, tool_input=tool_input, log=llm_output
        )

In [ ]:
output_parser = CustomOutputParser()

### Content Handlers

Content Handlers are defined to convert the input to the input expected by the model and to extract the response from the JSON returned by the model. Note that the LlamaContentHandler extracts the response from ```response_json[0]["generation"]``` while the ContentHandler (used by Falcon) extracts the response from ```return response_json[0]["generated_text"]```

In [ ]:
from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains.question_answering import load_qa_chain
import json


class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
#        return response_json["vectors"]
        return response_json[0]["generated_text"]



content_handler = ContentHandler()


In [ ]:
class LlamaContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
#        return response_json["vectors"]
        return response_json[0]["generation"]



content_handler = ContentHandler()
llama_content_handler = LlamaContentHandler()


### Define the LLMs

We define the LLMs that we want to use for predictions. Ensure that you change the ```endpoint_name``` and ```region``` to point to your own Sagemaker endpoint

In [ ]:
llm_falcon = SagemakerEndpoint(
endpoint_name="hf-llm-falcon-40b-bf16-2023-06-24-20-20-44-608",
model_kwargs={
     "parameters" : {"do_sample": False,
    "top_p": 0.9,
    "temperature": 0.1,
    "max_new_tokens": 40
              }},
region_name="us-east-1",
content_handler=content_handler
)



llm_llama = SagemakerEndpoint(
endpoint_name="jumpstart-dft-meta-textgeneration-llama-2-7b",
model_kwargs={
     "parameters" : {"return_full_text": False,
    "top_p": 0.9,
    "temperature": 0.1,
    "max_new_tokens": 40
              }},
endpoint_kwargs={"CustomAttributes": "accept_eula=true"},
region_name="us-east-1",
content_handler=llama_content_handler
)





### Define the LLM Chain

The LLM chain is defined and we provide the LLM that we want to use along with the PromptTemplate that we want to use

In [ ]:
llm_chain = LLMChain(llm=llm_llama, prompt=falcon_prompt)
#falcon_llm_chain = LLMChain(llm=llm_falcon, prompt=prompt)

### Define and create the Agent

In this implementation we use the LLMSingleActionAgent (https://js.langchain.com/docs/api/agents/classes/LLMSingleActionAgent) as this is a simplistic tool that generates an output in a single action, i.e. Add 2 numbers gives an deterministic answer right away. However, if your agents are more sophisticated and require multiple calls to the tool to derive an answer, you can choose to use another agent 

In [ ]:
tool_names = [tool.name for tool in tools]

agent = LLMSingleActionAgent(
    #llm_chain=llm_chain,
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nHuman:"],
    allowed_tools=tool_names,
)



In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

### Inference

Now that we have defined the tools, the prompts and the agent, we can perform an inference. When you run the block below you should see an output like 

```
> Entering new  chain...
 Thought: I need to use a tool to add two numbers 
 Action: AdditionTool() 
 Action Input: 10,36 
 Observation: the sum is
the add numbers tools executed




Human:The sum is 46

Action: Finish[46]

> Finished chain.
'[46]'
```

A debug statement in the tool ```the add numbers tools executed``` confirms that the tool was invoked and used to derive the answer

In [ ]:
langchain.debug = False
agent_executor.run("sum of 10 and 36")

In [ ]:
agent_executor.run("get the difference between 500 and 10")

In [ ]:
langchain.debug = False
agent_executor.run("Generate a number between 0 and 500000")